In [1]:
import re
import xml.etree.ElementTree as ET
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn
#from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
tree = ET.parse('ABSA-15_Restaurants_Train_Final.xml')
root = tree.getroot()

In [2]:
skip = False
words = []
vecs = []
with open("fasttext150", "r") as f:
    for line in f:
        if skip:
            info = line.strip().split(' ')
            words.append(info[0]) 
            c = []
            for i in range(1, len(info)):
                c.append(float(info[i]))
            vecs.append(c)
        else:
            skip = True
npvecs = np.asarray(vecs, dtype='float32')

In [3]:
word_id = {}

vecs_lookup = np.concatenate([np.zeros((1, 100), dtype='float32'), npvecs], axis=0)
_id = 1

for word in words:
    if word not in word_id:
        word_id[word] = _id
        
        _id += 1



In [4]:
sents = []
opins = []
for review in root:
    for sentence in review[0]:
        #sentence[0] #text
        if len(sentence) > 1:
            _ = re.sub(r"[\.\-\?\!,\"\']", " ", sentence[0].text.lower()).strip().split(' ')
            __ = []
            for word in _:
                if word != "":#get rid of ""s
                    #__.append(word)
                    if word in word_id:
                        __.append(word_id[word])
                    else:
                        __.append(0)
            if len(__) != 0:
                sents.append(__)
                opinions = [opin.attrib for opin in sentence[1]]
                opins.append(opinions)



In [5]:
_ = sorted(zip(sents,opins), key=lambda pair: len(pair[0]))
sents, opins = zip(*_)

In [6]:
classes_labels = {'positive':1, 'negative':-1, 'neutral':0}
dumb_Y = []
for opin  in opins:
    _ = [target_op['polarity'] for target_op in opin]
    __ = np.asarray([classes_labels[class_] for class_ in _], dtype = 'int32')
    dumb_Y.append(round(np.mean(__, dtype = 'int32'))) #labels are crushed into a mean
        

In [7]:
def batch_maker(sents, dumb_Y, min_samples, sample_size, pos):
    #sample size not fixed!!!!!!!!!!!!!!
    batch = []
    y_batch = []
    
    while len(batch) < min_samples and pos < len(dumb_Y): #not really
        z = [0,0,0]
        #print("i got here pos = ", pos)
        #print(len(dumb_Y))
        index_to_set_1 = dumb_Y[pos]
        z[1+index_to_set_1] = 1 #one-hot dumb_Y prepared for ith sent
        
        if len(sents[pos]) <= sample_size:
            _ = sents[pos][0:]
            #while len(_) !=  sample_size:
            #    _.append(0)
            if len(_) != sample_size:
                _ = np.concatenate((_, np.zeros(sample_size - len(_), dtype = "int32")))
            y_batch.append(z)
            batch.append(np.asarray(_, dtype = "int32"))
            
            pos += 1 #sentence processed
        else:
            counter = 0
            while counter*sample_size < len(sents[pos]):
                _ = sents[pos][counter*sample_size:(counter+1)*sample_size]
                #while len(_) !=  sample_size:
                #    _.append(0)
                if len(_) != sample_size:
                    _ = np.concatenate((_, np.zeros(sample_size - len(_), dtype = "int32")))
                counter += 1
                y_batch.append(z)
                batch.append(np.asarray(_, dtype = "int32"))
                
            pos += 1 #sentence processed
    batch = np.asarray(batch)
    y_batch = np.asarray(y_batch)
    return batch, y_batch, pos

In [8]:
train_amount = round(len(dumb_Y)*0.8)
train_sents = sents[:train_amount]
train_Y = dumb_Y[:train_amount]
test_sents = sents[train_amount:]
test_Y = dumb_Y[train_amount:]
print(len(train_Y), len(train_sents))

896 896


a,b,c = batch_maker(sents, dumb_Y, 10, 15, 850)
print(a,b,c)
print(sents[850:855], dumb_Y[850:855])
print(len(sents[1119]))

In [9]:
# Training Parameters
learning_rate = 0.001
training_steps = 10

display_step = 1

# Network Parameters
num_input =  100
timesteps = 40 # timesteps ~ words
num_hidden = 128 # hidden layer num of features
num_classes = 3

X = tf.placeholder("int32", [None, timesteps])
Y = tf.placeholder("int32", [None, num_classes]) #one hot repr

In [10]:
word_embs = tf.constant(vecs_lookup, dtype=tf.float32)

In [11]:
n_filters = 20
width = 3

In [12]:
xavier_init = tf.contrib.layers.xavier_initializer()
filters = tf.Variable(xavier_init([width, num_input, n_filters])) #Xavier?

In [13]:
def neural_net(x):
    #print(tf.shape(x))
    x = tf.nn.embedding_lookup(word_embs, x) #embedding - returns batch x len x emb ????
    #print(tf.shape(x))
    x = tf.layers.dropout(inputs = x, rate = 0.2)

    x = tf.nn.conv1d(value=x, filters = filters, stride = 1,padding = 'VALID')
    x = tf.nn.relu(features = x)
    #print(tf.shape(x))
    x = tf.layers.max_pooling1d(inputs = x, pool_size = timesteps - 2, strides = 1, padding = 'VALID') #pool_size = tf.shape(x)[1]
    #print(tf.shape(x))
    x = tf.squeeze(x, [1]) #because of one fictional dim from global max pooling
    x = tf.layers.dense(inputs = x, units = 250, activation = 'relu') #has his variables inside?
    x = tf.layers.dropout(inputs = x, rate = 0.2)
    #print(tf.shape(x))
    x = tf.layers.dense(inputs = x, units = 3) #has his variables inside? softmax goes there later
    #print(tf.shape(x))
    return x


In [14]:
logits = neural_net(X)

#this WAS in LSTM. needed?
prediction = tf.nn.softmax(logits) #so, another softmax on the output


# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y)) #there is softmax applied to the last output
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

output_labels = tf.argmax(logits, 1)


# Evaluate model (with test logits, for dropout to be disabled)
#print('\n',tf.shape(prediction), tf.shape(tf.argmax(prediction, 1)), tf.shape(tf.argmax(Y, 1)))
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))#tf.argmax(logits, 1),


answers = tf.argmax(prediction, 1)
true_labels = tf.argmax(Y, 1)


accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



TP(-1) = количество верно обнаруженных negative
FP(-1) = количество неверно обнаруженных negative (система говорит negative)
FN(-1) = количество неверно не обнаруженных negative (система говорит не negative)
для 0 и +1 аналогично

Precision(-1) = TP(-1) / (TP(-1) + FP(-1))
Recall(-1) = TP(-1) / (TP(-1) + FN(-1))
F-1(-1) = 2 * Precision(-1) * Recall(-1) / (Precision(-1) + Recall(-1))

Macro F-1 = (F-1(-1) + F-1(0) + F-1(+1)) / 3
Micro F-1 = P(-1)*F-1(-1) + P(0)*F-1(0) + P(+1)*F-1(+1)

In [15]:
sess = tf.InteractiveSession()
sess.run(init)

#print(len(train_Y))
for step in range(1, training_steps+1):
    i = 0
    while i < len(train_sents):
        x_batch, y_batch, i = batch_maker(train_sents, train_Y, min_samples = 32, sample_size = timesteps, pos = i)
        #print(i, x_batch.shape, y_batch.shape)
        #x_lookup = tf.nn.embedding_lookup(word_embs, x_batch)
        #sess.run(x_lookup)
        sess.run(train_op, feed_dict={X: x_batch, Y: y_batch}) #x_lookup
        #print('HUY')
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: x_batch, Y: y_batch}) #x_lookup
            
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")


Step 1, Minibatch Loss= 1.0638, Training Accuracy= 0.688
Step 1, Minibatch Loss= 0.9886, Training Accuracy= 0.844
Step 1, Minibatch Loss= 0.9595, Training Accuracy= 0.719
Step 1, Minibatch Loss= 0.8428, Training Accuracy= 0.906
Step 1, Minibatch Loss= 0.8188, Training Accuracy= 0.812
Step 1, Minibatch Loss= 0.9286, Training Accuracy= 0.656
Step 1, Minibatch Loss= 0.8650, Training Accuracy= 0.656
Step 1, Minibatch Loss= 0.6720, Training Accuracy= 0.812
Step 1, Minibatch Loss= 0.6879, Training Accuracy= 0.781
Step 1, Minibatch Loss= 0.7457, Training Accuracy= 0.750
Step 1, Minibatch Loss= 0.6233, Training Accuracy= 0.812
Step 1, Minibatch Loss= 1.1141, Training Accuracy= 0.531
Step 1, Minibatch Loss= 0.5828, Training Accuracy= 0.812
Step 1, Minibatch Loss= 0.7020, Training Accuracy= 0.750
Step 1, Minibatch Loss= 0.9362, Training Accuracy= 0.656
Step 1, Minibatch Loss= 0.9707, Training Accuracy= 0.688
Step 1, Minibatch Loss= 0.8783, Training Accuracy= 0.688
Step 1, Minibatch Loss= 0.7854,

Step 7, Minibatch Loss= 0.4250, Training Accuracy= 0.812
Step 7, Minibatch Loss= 0.9267, Training Accuracy= 0.562
Step 7, Minibatch Loss= 0.4316, Training Accuracy= 0.812
Step 7, Minibatch Loss= 0.4935, Training Accuracy= 0.781
Step 7, Minibatch Loss= 0.6564, Training Accuracy= 0.719
Step 7, Minibatch Loss= 0.7296, Training Accuracy= 0.656
Step 7, Minibatch Loss= 0.5939, Training Accuracy= 0.688
Step 7, Minibatch Loss= 0.5586, Training Accuracy= 0.844
Step 7, Minibatch Loss= 0.5529, Training Accuracy= 0.844
Step 7, Minibatch Loss= 0.9820, Training Accuracy= 0.625
Step 7, Minibatch Loss= 0.4038, Training Accuracy= 0.906
Step 7, Minibatch Loss= 0.5027, Training Accuracy= 0.844
Step 7, Minibatch Loss= 0.4422, Training Accuracy= 0.844
Step 7, Minibatch Loss= 0.4253, Training Accuracy= 0.812
Step 7, Minibatch Loss= 0.6901, Training Accuracy= 0.625
Step 7, Minibatch Loss= 0.9002, Training Accuracy= 0.625
Step 7, Minibatch Loss= 0.5184, Training Accuracy= 0.844
Step 7, Minibatch Loss= 0.4680,

In [20]:
all_answers = np.array([])
all_true_labels = np.array([])
i = 0

while i < len(test_sents):
    x_batch, y_batch, i = batch_maker(test_sents, test_Y, min_samples = 32, sample_size = timesteps, pos = i)
    
    a, t = sess.run([answers, true_labels], feed_dict={X: x_batch, Y: y_batch})
    #print(a,t)
    all_answers = np.append(all_answers, np.asarray(a)) #no need to flatten
    all_true_labels = np.append(all_true_labels, np.asarray(t))
    
    
TP, FP, FN = {0:0, 1:0, 2:0}, {0:0, 1:0, 2:0}, {0:0, 1:0, 2:0}
for l in range(3):
    for counter in range(len(all_answers)):
        if all_answers[counter] == all_true_labels[counter] and all_answers[counter] == l: #answer matches label
            TP[l] += 1
        if all_answers[counter] != all_true_labels[counter] and all_answers[counter] == l:
            FP[l] += 1
        if all_answers[counter] != all_true_labels[counter] and all_answers[counter] != l:
            FN[l] += 1
            
Prec, Rec = {}, {}
for l in range(3):
    try:
        Prec[l] = TP[l]/float((TP[l]+FP[l]))
    except Exception:
        Prec[l] = 0
    try:
        Rec[l] = TP[l]/float((TP[l]+FN[l]))
    except Exception:
        Rec[l] = 0
F1 = {}
for l in range(3):
    try:
        F1[l] = 2*Prec[l]*Rec[l]/(Prec[l]+Rec[l])
    except Exception:
        F1[l] = 0
    
    
MF1 = sum(_ for _ in F1.values())/3
print(F1, MF1)
#print(TP, FP, FN, Prec, Rec)


{0: 0.35374149659863946, 1: 0, 2: 0.6964856230031948} 0.35007570653394476


In [21]:
preds = np.array([])
i = 0
while i < len(test_sents):
    x_batch, y_batch, i = batch_maker(test_sents, test_Y, min_samples = 32, sample_size = timesteps, pos = i)
        #print(i, x_batch.shape, y_batch.shape)
        #x_lookup = tf.nn.embedding_lookup(word_embs, x_batch)
        #sess.run(x_lookup)
        #sess.run(train_op, feed_dict={X: x_batch, Y: y_batch}) #x_lookup
        #print('HUY')
             
    pred = sess.run(correct_pred, feed_dict={X: x_batch, Y: y_batch}) #x_lookup
    preds = np.append(preds, np.asarray(pred))
print(np.mean(preds))

0.5869565217391305


In [ ]:
sess.close()